In [2]:
import os
import time
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from tqdm import tqdm
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter

# ==== CONFIGURATION ====
DATA_DIR = r'C:\Users\USER\Downloads\data\skin\train'
WEIGHTS_ROOT = r'C:\Users\USER\Downloads\data\skin\weights'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 20
NUM_EPOCHS = 15
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 5e-4
NUM_CLASSES = 9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EARLY_STOP_PATIENCE = 5
GRAD_CLIP = 5.0
LOG_DIR = "runs/experiment"

# ==== AUGMENTATIONS ====
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(25),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.GaussianBlur(kernel_size=(5, 5)),
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),
    transforms.RandomPerspective(distortion_scale=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ==== DATA HANDLING ====
class SkinLesionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.samples = []
        
        for cls in self.classes:
            cls_dir = os.path.join(root_dir, cls)
            if os.path.isdir(cls_dir):
                for fname in os.listdir(cls_dir):
                    if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.samples.append((
                            os.path.join(cls_dir, fname),
                            self.class_to_idx[cls]
                        ))

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        return self.transform(img) if self.transform else img, label

# ==== MODEL ARCHITECTURE ====
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        base = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.features = nn.Sequential(*list(base.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x).flatten(1)
        return self.classifier(x)

# ==== TRAINING SETUP ====
def initialize_training():
    # Create unique training directory
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(WEIGHTS_ROOT, f"run-{timestamp}")
    os.makedirs(run_dir, exist_ok=True)

    # Dataset setup
    dataset = SkinLesionDataset(DATA_DIR)
    train_size = int(0.8 * len(dataset))
    train_set, val_set = random_split(dataset, [train_size, len(dataset)-train_size])
    train_set.dataset.transform = train_transform
    val_set.dataset.transform = val_transform

    # Class balancing
    class_counts = torch.tensor([
        sum(1 for s in dataset.samples if s[1] == i) 
        for i in range(len(dataset.classes))
    ])
    weights = (1.0 / class_counts.float()).to(DEVICE)
    weights = (weights / weights.sum()) * NUM_CLASSES

    # Model components
    model = ResNetClassifier(NUM_CLASSES).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

    return (
        DataLoader(train_set, BATCH_SIZE, shuffle=True, pin_memory=True),
        DataLoader(val_set, BATCH_SIZE*2, pin_memory=True),
        nn.CrossEntropyLoss(weight=weights),
        model,
        optimizer,
        scheduler,
        SummaryWriter(LOG_DIR),
        torch.cuda.amp.GradScaler(),
        run_dir,
        dataset.classes
    )

# ==== CORE TRAINING LOOP ====
def train():
    train_loader, val_loader, criterion, model, optimizer, scheduler, writer, scaler, run_dir, class_names = initialize_training()
    best_acc, no_improve = 0.0, 0

    for epoch in range(NUM_EPOCHS):
        # Training Phase
        model.train()
        train_loss, correct, total = 0.0, 0, 0
        for inputs, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        # Validation Phase
        val_loss, all_preds, all_labels = 0.0, [], []
        model.eval()
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                all_preds.extend(torch.argmax(outputs, 1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculate metrics
        train_acc = 100 * correct / total
        val_acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
        val_loss /= len(val_loader)
        scheduler.step(val_acc)

        # Reporting
        writer.add_scalars('Loss', {'train': train_loss/len(train_loader), 'val': val_loss}, epoch)
        writer.add_scalars('Accuracy', {'train': train_acc, 'val': val_acc}, epoch)
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f}")
        print(f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")
        print(classification_report(all_labels, all_preds, target_names=class_names))

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            no_improve = 0
            model_name = f"best_epoch{epoch+1}_acc{val_acc:.2f}.pth"
            save_path = os.path.join(run_dir, model_name)
            torch.save(model.state_dict(), save_path)
            print(f"🏆 New best model saved: {save_path}")
        else:
            no_improve += 1
            if no_improve >= EARLY_STOP_PATIENCE:
                print(f"\n⏹ Early stopping at epoch {epoch+1} with best accuracy {best_acc:.2f}%")
                break

    writer.close()
    print(f"\n✅ Training complete! Best validation accuracy: {best_acc:.2f}%")
    print(f"📁 All weights saved in: {run_dir}")

# Start training
train()

C:\Users\USER\AppData\Local\Temp\ipykernel_15824\3446584269.py:125: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  torch.cuda.amp.GradScaler(),
Train Epoch 1:   0%|          | 0/28 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_15824\3446584269.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 1: 100%|██████████| 4/4 [00:02<00:00,  1.59it/s]



Epoch 1/15
Train Loss: 1.5212 | Val Loss: 0.7889
Train Acc: 47.76% | Val Acc: 77.14%
                            precision    recall  f1-score   support

         Actinic keratosis       0.73      0.89      0.80         9
         Atopic Dermatitis       0.73      0.94      0.82        17
          Benign keratosis       0.95      0.95      0.95        20
            Dermatofibroma       0.71      0.79      0.75        19
         Melanocytic nevus       0.68      0.93      0.79        14
                  Melanoma       0.71      0.42      0.53        12
   Squamous cell carcinoma       0.70      0.44      0.54        16
Tinea Ringworm Candidiasis       1.00      0.54      0.70        13
           Vascular lesion       0.78      0.90      0.84        20

                  accuracy                           0.77       140
                 macro avg       0.78      0.75      0.75       140
              weighted avg       0.78      0.77      0.76       140

🏆 New best model saved: C:\

Train Epoch 2:   0%|          | 0/28 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_15824\3446584269.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 2: 100%|██████████| 4/4 [00:01<00:00,  3.63it/s]



Epoch 2/15
Train Loss: 0.7220 | Val Loss: 0.5413
Train Acc: 80.25% | Val Acc: 79.29%
                            precision    recall  f1-score   support

         Actinic keratosis       0.47      1.00      0.64         9
         Atopic Dermatitis       1.00      0.88      0.94        17
          Benign keratosis       0.90      0.95      0.93        20
            Dermatofibroma       0.75      0.79      0.77        19
         Melanocytic nevus       0.67      1.00      0.80        14
                  Melanoma       0.67      0.33      0.44        12
   Squamous cell carcinoma       0.75      0.19      0.30        16
Tinea Ringworm Candidiasis       0.93      1.00      0.96        13
           Vascular lesion       0.95      0.95      0.95        20

                  accuracy                           0.79       140
                 macro avg       0.79      0.79      0.75       140
              weighted avg       0.81      0.79      0.77       140

🏆 New best model saved: C:\

Train Epoch 3:   0%|          | 0/28 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_15824\3446584269.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 3: 100%|██████████| 4/4 [00:01<00:00,  3.98it/s]


...


Epoch 8/15
Train Loss: 0.0926 | Val Loss: 0.4020
Train Acc: 98.38% | Val Acc: 85.71%
                            precision    recall  f1-score   support

         Actinic keratosis       0.67      0.89      0.76         9
         Atopic Dermatitis       1.00      0.88      0.94        17
          Benign keratosis       0.86      0.95      0.90        20
            Dermatofibroma       0.94      0.79      0.86        19
         Melanocytic nevus       0.82      1.00      0.90        14
                  Melanoma       0.80      0.67      0.73        12
   Squamous cell carcinoma       0.69      0.56      0.62        16
Tinea Ringworm Candidiasis       0.87      1.00      0.93        13
           Vascular lesion       0.95      0.95      0.95        20

                  accuracy                           0.86       140
                 macro avg       0.84      0.85      0.84       140
              weighted avg       0.86      0.86      0.85       140



Train Epoch 9:   0%|          | 0/28 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_15824\3446584269.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 9: 100%|██████████| 4/4 [00:01<00:00,  3.95it/s]



Epoch 9/15
Train Loss: 0.0639 | Val Loss: 0.4147
Train Acc: 98.38% | Val Acc: 85.00%
                            precision    recall  f1-score   support

         Actinic keratosis       0.40      0.89      0.55         9
         Atopic Dermatitis       1.00      0.82      0.90        17
          Benign keratosis       1.00      0.95      0.97        20
            Dermatofibroma       0.94      0.89      0.92        19
         Melanocytic nevus       0.70      1.00      0.82        14
                  Melanoma       0.86      0.50      0.63        12
   Squamous cell carcinoma       0.89      0.50      0.64        16
Tinea Ringworm Candidiasis       1.00      1.00      1.00        13
           Vascular lesion       1.00      1.00      1.00        20

                  accuracy                           0.85       140
                 macro avg       0.87      0.84      0.83       140
              weighted avg       0.90      0.85      0.85       140



Train Epoch 10:   0%|          | 0/28 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_15824\3446584269.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 10: 100%|██████████| 4/4 [00:01<00:00,  3.84it/s]


Epoch 10/15
Train Loss: 0.0501 | Val Loss: 0.5520
Train Acc: 99.46% | Val Acc: 82.86%
                            precision    recall  f1-score   support

         Actinic keratosis       0.39      1.00      0.56         9
         Atopic Dermatitis       1.00      0.82      0.90        17
          Benign keratosis       1.00      0.95      0.97        20
            Dermatofibroma       0.94      0.89      0.92        19
         Melanocytic nevus       0.70      1.00      0.82        14
                  Melanoma       0.83      0.42      0.56        12
   Squamous cell carcinoma       0.83      0.31      0.45        16
Tinea Ringworm Candidiasis       0.93      1.00      0.96        13
           Vascular lesion       1.00      1.00      1.00        20

                  accuracy                           0.83       140
                 macro avg       0.85      0.82      0.80       140
              weighted avg       0.88      0.83      0.82       140


⏹ Early stopping at epoch

In [ ]:
import os
import time
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from tqdm import tqdm
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter

# ==== CONFIGURATION ====
DATA_DIR = r'C:\Users\USER\Downloads\data\skin\trainn'
WEIGHTS_ROOT = r'C:\Users\USER\Downloads\data\skin\weights'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 20
NUM_EPOCHS = 15
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 5e-4
NUM_CLASSES = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EARLY_STOP_PATIENCE = 5
GRAD_CLIP = 5.0
LOG_DIR = "runs/experiment"

# ==== AUGMENTATIONS ====
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(25),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.GaussianBlur(kernel_size=(5, 5)),
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),
    transforms.RandomPerspective(distortion_scale=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ==== DATA HANDLING ====
class SkinLesionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.samples = []
        
        for cls in self.classes:
            cls_dir = os.path.join(root_dir, cls)
            if os.path.isdir(cls_dir):
                for fname in os.listdir(cls_dir):
                    if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.samples.append((
                            os.path.join(cls_dir, fname),
                            self.class_to_idx[cls]
                        ))

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        return self.transform(img) if self.transform else img, label

# ==== MODEL ARCHITECTURE ====
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        base = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.features = nn.Sequential(*list(base.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x).flatten(1)
        return self.classifier(x)

# ==== TRAINING SETUP ====
def initialize_training():
    # Create unique training directory
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(WEIGHTS_ROOT, f"run-{timestamp}")
    os.makedirs(run_dir, exist_ok=True)

    # Dataset setup
    dataset = SkinLesionDataset(DATA_DIR)
    train_size = int(0.8 * len(dataset))
    train_set, val_set = random_split(dataset, [train_size, len(dataset)-train_size])
    train_set.dataset.transform = train_transform
    val_set.dataset.transform = val_transform

    # Class balancing
    class_counts = torch.tensor([
        sum(1 for s in dataset.samples if s[1] == i) 
        for i in range(len(dataset.classes))
    ])
    weights = (1.0 / class_counts.float()).to(DEVICE)
    weights = (weights / weights.sum()) * NUM_CLASSES

    # Model components
    model = ResNetClassifier(NUM_CLASSES).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

    return (
        DataLoader(train_set, BATCH_SIZE, shuffle=True, pin_memory=True),
        DataLoader(val_set, BATCH_SIZE*2, pin_memory=True),
        nn.CrossEntropyLoss(weight=weights),
        model,
        optimizer,
        scheduler,
        SummaryWriter(LOG_DIR),
        torch.cuda.amp.GradScaler(),
        run_dir,
        dataset.classes
    )

# ==== CORE TRAINING LOOP ====
def train():
    train_loader, val_loader, criterion, model, optimizer, scheduler, writer, scaler, run_dir, class_names = initialize_training()
    best_acc, no_improve = 0.0, 0

    for epoch in range(NUM_EPOCHS):
        # Training Phase
        model.train()
        train_loss, correct, total = 0.0, 0, 0
        for inputs, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        # Validation Phase
        val_loss, all_preds, all_labels = 0.0, [], []
        model.eval()
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                all_preds.extend(torch.argmax(outputs, 1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculate metrics
        train_acc = 100 * correct / total
        val_acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
        val_loss /= len(val_loader)
        scheduler.step(val_acc)

        # Reporting
        writer.add_scalars('Loss', {'train': train_loss/len(train_loader), 'val': val_loss}, epoch)
        writer.add_scalars('Accuracy', {'train': train_acc, 'val': val_acc}, epoch)
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f}")
        print(f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")
        print(classification_report(all_labels, all_preds, target_names=class_names))

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            no_improve = 0
            model_name = f"best_epoch{epoch+1}_acc{val_acc:.2f}.pth"
            save_path = os.path.join(run_dir, model_name)
            torch.save(model.state_dict(), save_path)
            print(f"🏆 New best model saved: {save_path}")
        else:
            no_improve += 1
            if no_improve >= EARLY_STOP_PATIENCE:
                print(f"\n⏹ Early stopping at epoch {epoch+1} with best accuracy {best_acc:.2f}%")
                break

    writer.close()
    print(f"\n✅ Training complete! Best validation accuracy: {best_acc:.2f}%")
    print(f"📁 All weights saved in: {run_dir}")

# Start training
train()

C:\Users\USER\AppData\Local\Temp\ipykernel_38244\3934271191.py:125: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  torch.cuda.amp.GradScaler(),
Train Epoch 1:   0%|          | 0/25 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_38244\3934271191.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 1: 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]



Epoch 1/15
Train Loss: 1.4066 | Val Loss: 0.7003
Train Acc: 54.36% | Val Acc: 79.84%
                            precision    recall  f1-score   support

         Actinic keratosis       0.73      0.94      0.82        17
         Atopic Dermatitis       0.93      0.93      0.93        14
          Benign keratosis       0.82      0.70      0.76        20
            Dermatofibroma       1.00      0.39      0.56        18
         Melanocytic nevus       0.83      0.94      0.88        16
   Squamous cell carcinoma       0.71      0.71      0.71        14
Tinea Ringworm Candidiasis       1.00      1.00      1.00        13
           Vascular lesion       0.58      0.92      0.71        12

                  accuracy                           0.80       124
                 macro avg       0.83      0.82      0.80       124
              weighted avg       0.83      0.80      0.79       124

🏆 New best model saved: C:\Users\USER\Downloads\data\skin\weights\run-20250420-234143\best_epoc

Train Epoch 2:   0%|          | 0/25 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_38244\3934271191.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 2: 100%|██████████| 4/4 [00:01<00:00,  2.04it/s]



Epoch 2/15
Train Loss: 0.5771 | Val Loss: 0.4532
Train Acc: 85.40% | Val Acc: 85.48%
                            precision    recall  f1-score   support

         Actinic keratosis       0.78      0.82      0.80        17
         Atopic Dermatitis       1.00      0.79      0.88        14
          Benign keratosis       1.00      1.00      1.00        20
            Dermatofibroma       1.00      0.67      0.80        18
         Melanocytic nevus       0.94      0.94      0.94        16
   Squamous cell carcinoma       0.55      0.79      0.65        14
Tinea Ringworm Candidiasis       0.87      1.00      0.93        13
           Vascular lesion       0.83      0.83      0.83        12

                  accuracy                           0.85       124
                 macro avg       0.87      0.85      0.85       124
              weighted avg       0.88      0.85      0.86       124

🏆 New best model saved: C:\Users\USER\Downloads\data\skin\weights\run-20250420-234143\best_epoc

Train Epoch 3:   0%|          | 0/25 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_38244\3934271191.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 3: 100%|██████████| 4/4 [00:02<00:00,  1.89it/s]


...

Train Epoch 14:   0%|          | 0/25 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_38244\3934271191.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 14: 100%|██████████| 4/4 [00:01<00:00,  2.72it/s]



Epoch 14/15
Train Loss: 0.0177 | Val Loss: 0.2462
Train Acc: 99.59% | Val Acc: 92.74%
                            precision    recall  f1-score   support

         Actinic keratosis       0.81      1.00      0.89        17
         Atopic Dermatitis       1.00      0.93      0.96        14
          Benign keratosis       1.00      1.00      1.00        20
            Dermatofibroma       0.94      0.83      0.88        18
         Melanocytic nevus       1.00      1.00      1.00        16
   Squamous cell carcinoma       0.77      0.71      0.74        14
Tinea Ringworm Candidiasis       0.93      1.00      0.96        13
           Vascular lesion       1.00      0.92      0.96        12

                  accuracy                           0.93       124
                 macro avg       0.93      0.92      0.93       124
              weighted avg       0.93      0.93      0.93       124



Train Epoch 15:   0%|          | 0/25 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_38244\3934271191.py:143: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation Epoch 15: 100%|██████████| 4/4 [00:01<00:00,  2.82it/s]


Epoch 15/15
Train Loss: 0.0137 | Val Loss: 0.2566
Train Acc: 99.80% | Val Acc: 91.13%
                            precision    recall  f1-score   support

         Actinic keratosis       0.79      0.88      0.83        17
         Atopic Dermatitis       1.00      0.93      0.96        14
          Benign keratosis       1.00      1.00      1.00        20
            Dermatofibroma       1.00      0.78      0.88        18
         Melanocytic nevus       1.00      1.00      1.00        16
   Squamous cell carcinoma       0.65      0.79      0.71        14
Tinea Ringworm Candidiasis       0.93      1.00      0.96        13
           Vascular lesion       1.00      0.92      0.96        12

                  accuracy                           0.91       124
                 macro avg       0.92      0.91      0.91       124
              weighted avg       0.92      0.91      0.91       124


⏹ Early stopping at epoch 15 with best accuracy 93.55%

✅ Training complete! Best validation 

: 